## Design of yeast/E. coli shuttle vector pTA1

The pTA1 was made by in-vivo homologous recombination between five DNA fragments 
carrying 30 bp long flanking recombination sequences (s1 - s5).

              s1
     -|ampR|30
    |       \/                                           
    |       /\       s2                                  
    |       30|pBR|30
    |              \/
    |              /\        s3
    |              30|_2µ_|30
    |                      \/
    |                      /\        s4
    |                      30|LEU2|30
    |                              \/
    |                              /\        s5
    |                              30|ΔCRP|30
    |                                      \/
    |                                      /\
    |                                      30-
    |                                         |
     -----------------------------------------

| Designation | Sequence                       |
|-------------|--------------------------------|
| s1          | AATCCAATCAGCGTAAGGTGTAGACTTTCT |
| s2          | ATCGTATCTGCTGCGTAAATAGTAGTCAAC |
| s3          | TAAAATCTCGTAAAGGAACTGTCTGCTCTG |
| s4          | AACTGTAAAATCAGGTATCTCGTAGTCCGT |
| s5          | GAAAAGCGTTTACCTCGGAACTCTATTGTA |



| Primer             | Sequence                                                  |
|--------------------|-----------------------------------------------------------|
| 980_Leu2.FW        | TAAAATCTCGTAAAGGAACTGTCTGCTCTGTTAACTGTGGGAATACTC          |
| 979_Leu2.REV       | ACGGACTACGAGATACCTGATTTTACAGTTTCTACCCTATGAACATATTC        |
| 1113_Amp.fw.nw     | GAAAAGCGTTTACCTCGGAACTCTATTGTAGAACCCCTATTTGTTTATTTTTCTA   |
| 987_Amp.REV        | AGAAAGTCTACACCTTACGCTGATTGGATTTGAAGTTTTAAATCAATCTAAA      |
| 1196_Pbr.FW        | AATCCAATCAGCGTAAGGTGTAGACTTTCTCTGTCAGACCAAGTTTACT         |
| 1195_Pbr.REV       | GTTGACTACTATTTACGCAGCAGATACGATCTCGTTTCATCGGT              |
| 978_Crp.FW         | AACTGTAAAATCAGGTATCTCGTAGTCCGTGTTCTGATCCTCGAGC            |
| 977_Crp.REV        | TACAATAGAGTTCCGAGGTAAACGCTTTTCGTTCTTGTCTCATTGCC           |
| 984_Micron.FW      | ATCGTATCTGCTGCGTAAATAGTAGTCAACGATCGTACTTGTTACCCAT         |
| 983_Micron.REV     | CAGAGCAGACAGTTCCTTTACGAGATTTTAGATCCAATATCAAAGGAA          |



In [1]:
from pydna.readers import read
from pydna.amplify import pcr
from pydna.assembly import Assembly
from pydna.myprimers import PrimerList
p = PrimerList()

In [2]:
from io import StringIO

In [3]:
from markdown_it import MarkdownIt
md = MarkdownIt().enable('table')
from pandas import read_html

text = ("""\
| Primer1 | Primer2 | Target | Template          |
|---------|---------|--------|-------------------|
| 1113    | 987     | ampR   | ampR/pBR322.gb    |
| 1196    | 1195    | pBR    | pbr/pBR322.gb     |
| 984     | 983     | 2µ     | 2µ/YEplac112.gb   |
| 980     | 979     | LEU2   | LEU2/YEplac181.gb |
| 978     | 977     | CRP∆   | crp/pYPKpw.gb     |

""")

md.parse(text)
dfrm, *rest = read_html(md.render(text))
dfrm

,Primer1,Primer2,Target,Template
0,1113,987,ampR,ampR/pBR322.gb
1,1196,1195,pBR,pbr/pBR322.gb
2,984,983,2µ,2µ/YEplac112.gb
3,980,979,LEU2,LEU2/YEplac181.gb
4,978,977,CRP∆,crp/pYPKpw.gb


In [4]:
products = []
for r, row in dfrm.iterrows():
    fp = p[int(row["Primer1"])]
    rp = p[int(row["Primer2"])]
    template = read(row["Template"])
    product = pcr(fp, rp, template)
    product.name = row["Target"]
    products.append(product)

In [5]:
mcs = product

In [6]:
product.name

'CRP∆'

In [7]:
products

[Amplicon(1072), Amplicon(1497), Amplicon(1644), Amplicon(1667), Amplicon(445)]

In [8]:
asm = Assembly(products)

In [9]:
asm

Assembly
fragments..: 1072bp 1497bp 1644bp 1667bp 445bp
limit(bp)..: 25
G.nodes....: 12
algorithm..: common_sub_strings

In [10]:
candidates = asm.assemble_circular()

In [11]:
candidates

[Contig(o6175), Contig(o6175), Contig(o6095), Contig(o6095), Contig(o80)]

In [12]:
candidate, *rest = candidates

In [13]:
candidate.figure()

 -|ampR|30
|       \/
|       /\
|       30|CRP∆_rc|30
|                  \/
|                  /\
|                  30|LEU2_rc|30
|                             \/
|                             /\
|                             30|2µ_rc|30
|                                      \/
|                                      /\
|                                      30|pBR|30
|                                             \/
|                                             /\
|                                             30-
|                                                |
 ------------------------------------------------

In [14]:
len(candidate)

6175

In [15]:
pTA1 = candidate.synced("gttctgatcctcgagcatcttaagaattc")

In [16]:
pTA1.stamp()

cSEGUID T61GO_ZENT_hjleUKIMiK8XTlUM

In [17]:
pTA1.name = "pTA1"

In [18]:
pTA1.description = "pTA1 ampR pBR 2µ LEU2 ΔCRP µ828"

In [19]:
mcs[30:-30].cutters( pTA1.unique_cutters() )

RestrictionBatch(['AatII', 'AleI', 'BalI', 'BclI', 'EcoRI', 'FbaI', 'FspAI', 'Ksp22I', 'MlsI', 'MluNI', 'Mox20I', 'MscI', 'Msp20I', 'OliI', 'PaeI', 'PaeR7I', 'PspXI', 'Sfr274I', 'SlaI', 'SphI', 'XbaI', 'XhoI', 'ZraI'])

In [20]:
pTA1.write("pTA1.gb")

pTA1.gb